# Ignore files

This page specifies aspects of how exactly `.dockerignore` works.

---

The following cell creates:

- `dockerfile` that tries to copy `folder` and pring it's content during container start.
- `.dockerignore` that counts `folder`.
- And `folder` that contains `file`.

In [ ]:
mkdir folder
echo "message" > folder/file

# .dockerignore
cat << EOF > .dockerignore
folder
EOF

# dockerfile
cat << EOF > dockerfile
FROM alpine
COPY folder folder
CMD ["ls", "folder"]
EOF

If you try to build an image from such a configuration, you'll get an error - `"/folder": not found` in the `COPY` step. This happens because `.dockerignore` has banned this file.

In [ ]:
docker build -t test_image .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (6/6) FINISHED                                 docker:default
 => [internal] load build definition from dockerfile                       0.0s
 => => transferring dockerfile: 89B                                        0.0s
 => [internal] load metadata for docker.io/library/alpine:latest           0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 47B                                           0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [1/2] FROM docker.io/library/alpine:latest                             0.0s
 => ERROR [2/2] COPY folder folder                                         0.0s
------
 > [2/2] COPY folder folder:
------
dockerfile:2
--------------------
   1 |     FROM alpine
   2 | >>> COPY fold

: 1

Ok, let's fix `.dockerignore` to reject `folder/ignored_file`. Add such a file, and another `folder/new_file`.

In [ ]:
cat << EOF > .dockerignore
folder/ignored_file
EOF
echo "message" > folder/ignored_file
echo "message" > folder/new_file

Now the image has been successfully built. And in the output of the container you can see `new_file`, but not `ignored_file`, which was rejected by `.dockerignore`.

In [ ]:
docker build -t test_image . &> /dev/null
docker run --rm test_image

file
new_file


Don't forget to keep your environment clean.

In [ ]:
rm -r .dockerignore dockerfile folder
docker rmi test_image

Untagged: test_image:latest
Deleted: sha256:35bbd1334f9243483ff527d8bee8594f18a50636c8d02ba39e0111c3393db3bc
